# Grove Analog ports and PWM

The arduino shield has dedicated analog ports which can be accessed through the `analog.h` header which is included by `mbio.h`. This driver can only be used with the A1 to A4 ports on the PYNQ shield and not at all using the PMOD interface. First we'll load the library and initialise the base overlay.

In [1]:
import ipython_microblaze as ipmb

from pynq.overlays.base import BaseOverlay
base = BaseOverlay('base.bit')

Following the other APIs the Analog API consists of the following functions

```C
analog analog_open_iop_grove(unsigned char port, unsigned char wire);
analog analog_open_iop_pin(unsigned char pin);

analog analog_open(int channel);
int analog_read_raw(analog);
float analog_read(analog);

int analog_range(analog);
float analog_vref(analog);
```

This peripheral follows the format of many in that upon connecting the pin, a handle is returned which can be used for reading the values back. `analog_read_raw` returns a number between 0 and MAX where MAX can be found using `analog_range`. `analog_read` will do the conversion to the voltage for you.

For a simple example, let's create a program which reads the value from the A1 port and sends it back to python.

In [2]:
%%microblaze_functions base.ARDUINO
#include <iop.h>

int analog_test() {
    analog conn = analog_open_iop_grove(A1, 0);
    int val = analog_read_raw(conn);
    return val;
}

In [3]:
import struct
print(analog_test())

analog_test.reset()

64978


Next we can introduce the timer and PWM API which generates a regular stream of pulses on an output pin.

```C
timer timer_open_iop_pin(unsigned char pin);
timer timer_open_iop_grove(unsigned char port, unsigned char wire);

timer timer_open(int index);
void timer_delay_us(timer, int channel, int usdelay);
void timer_pwm_generate(timer, int pulse, int period);
void timer_pwm_stop(timer);
```

This driver uses the same idiom as the `analog` driver. The PWM has some limitations to be aware of - for the Arduino output it can only be connected to digital pins D3, D5, D6, D9, D10 and D11 however all of these outputs can be used in parallel. For the PYNQ shield this restricts single-wire Grove peripherals to the G2, G4 and G6 connectors.

The `timer_pwm_generate` function starts the PWM generator and set the period and pulse width measured in cycles. `timer_pwm_stop` drives the output at 0.

For an example we can connect a sensor to A1 and an output such as an LED to G2 and use that to represent the current value of the input.

In [4]:
%%microblaze_functions base.ARDUINO
#include <iop.h>

void pwm_test() {
    analog input = analog_open_iop_grove(A1, 0);
    timer output = timer_open_iop_grove(G2, 0);
    int range = analog_range(input);
    int val = 0;
    while (1) {
        val = analog_read_raw(input);
        timer_pwm_generate(output, val, range);
    }
}

In [5]:
pwm_test()

The PWM library can also be used on a PMOD output although here there is only a single generator but it can be routed to any of the output pins. This example ramps an LED attached to G2

In [6]:
%%microblaze_functions base.PMODA
#include <iop.h>

void pwm_test_pmod() {
    timer output = timer_open_iop_grove(G2, 0);
    while (1) {
        for (int i = 0; i < 100; ++i) {
            timer_pwm_generate(output, i, 100);
            for (int j = 0; j < 1000000; ++j);
        }
    }
}

In [7]:
pwm_test_pmod()